# LangChain Expression Language (LCEL) Deepdive
LangChain Expression Language (LCEL) offers several key advantages over legacy chains:

1. **Composability**: LCEL allows you to easily combine different components using the | operator, making chain creation more intuitive and readable
2. **Type Safety**: LCEL provides better type checking and validation between components, reducing runtime errors
3. **Streaming Support**: Built-in streaming capabilities that work consistently across different components
4. **Flexible Input/Output**: Components can handle various input/output types and automatically convert between them when possible
5. **Better Testing**: The modular nature makes it easier to test individual components and mock dependencies
6. **Runtime Optimization**: LCEL can automatically optimize execution paths and handle caching more efficiently

In [1]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

/Users/thann/Library/CloudStorage/OneDrive-IndianInstituteofScience/Course_Contents/Deep Learning/Class Materials/Tutorials/deep-learning/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
model = ChatOpenAI(model="gpt-4o-mini")
output_parser = StrOutputParser() # This is a custom class that is used to parse the output of the model.

chain = prompt | model | output_parser

# Pipe in python is __or__ and __ror__ dunder methods, which are bitwise operators used for overloading. 
# Overloading in Python refers to the ability to have multiple methods and custom classes with the same name but different implementations.

chain.invoke({"topic": "ice cream"})

'What did the ice cream cone say to the scoop? \n\n“I’m so glad we’re together, I’m just so cone-fident about our relationship!”'

Each component in the chain is a callable/runnable object that can be chained together using the | operator

In [4]:
prompt.invoke({"topic": "ice cream"})

ChatPromptValue(messages=[HumanMessage(content='tell me a short joke about ice cream', additional_kwargs={}, response_metadata={})])

In [5]:
from langchain_core.messages.human import HumanMessage

messages = [HumanMessage(content='tell me a short joke about ice cream')]
model.invoke(messages)

AIMessage(content='Why did the ice cream truck break down?  \n\nBecause it had a rocky road!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 15, 'total_tokens': 32, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BeLGiDTHxcWfN8ekK3ion7GzO2iPL', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--3e8ca0af-022b-4df6-8c6c-14d78f9145ee-0', usage_metadata={'input_tokens': 15, 'output_tokens': 17, 'total_tokens': 32, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## CRunnable

1. **Abstract Base Class Pattern**:
    - `CRunnable` is an abstract class that defines a common interface for processing data. 
    - Any subclass must implement the `process()` method, ensuring consistent behavior across different runnable objects.

2. **Chain of Responsibility Implementation**:
    - Each `CRunnable` can be linked to another through the `next` attribute. 
    - When `invoke()` is called, it processes the data and automatically passes the result to the next runnable in the chain if one exists.

3. **Pipe Operator Overloading**:
    - The `__or__` method allows you to chain runnables using the `|` operator (e.g., `runnable1 | runnable2`).
    - Creating a more readable syntax for building processing pipelines.

4. **Sequence Wrapper Class**:
    - `CRunnableSequence` is a specialized runnable that wraps two other runnables, executing them in sequence. 
    - It bypasses the normal `process()` method and directly chains the `invoke()` calls from first to second runnable.


In [6]:
from abc import ABC, abstractmethod

class CRunnable(ABC): # CRunnable is an abstract class that defines the interface for a runnable object.
    def __init__(self):
        self.next = None # next is the argument to the constructor of the class.

    @abstractmethod # This is a decorator that ensures that the method is implemented by subclasses.
    def process(self, data):
        """
        This method must be implemented by subclasses to define
        data processing behavior.
        """
        pass

    def invoke(self, data):
        processed_data = self.process(data)
        if self.next is not None:
            return self.next.invoke(processed_data)
        return processed_data

    def __or__(self, other):
        return CRunnableSequence(self, other)
    
class CRunnableSequence(CRunnable):
    def __init__(self, first, second):
        super().__init__()
        self.first = first
        self.second = second

    def process(self, data):
        pass

    def invoke(self, data):
        first_result = self.first.invoke(data)
        return self.second.invoke(first_result)

In [7]:
class AddTen(CRunnable): # CRunnable is inherited from the abstract class ABC.
    def process(self, data):
        print("AddTen: ", data)
        return data + 10

class MultiplyByTwo(CRunnable):
    def process(self, data):
        print("Multiply by 2: ", data)
        return data * 2

class ConvertToString(CRunnable):
    def process(self, data):
        print("Convert to string: ", data)
        return f"Result: {data}"

In [8]:
a = AddTen()
b = MultiplyByTwo()
c = ConvertToString()

chain = a | b | c

In [9]:
result = chain.invoke(10)
print(result)

AddTen:  10
Multiply by 2:  20
Convert to string:  40
Result: 40


## Runnables from LangChain

In [10]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda, RunnableParallel

In [11]:
# RunnablePassthrough does nothing but pass the input through to the next runnable.
chain = RunnablePassthrough() | RunnablePassthrough () | RunnablePassthrough ()
chain.invoke("hello")

'hello'

In [12]:
def input_to_upper(input: str):
    output = input.upper()
    return output

In [13]:
chain = RunnablePassthrough() | RunnableLambda(input_to_upper) | RunnablePassthrough()
chain.invoke("hello")

'HELLO'

In [14]:
chain = RunnableParallel({"x": RunnablePassthrough(), "y": RunnablePassthrough()})

In [15]:
chain.invoke("hello")

{'x': 'hello', 'y': 'hello'}

In [ ]:
chain.invoke({"input": "hello", "input2": "goodbye"})

{'x': {'input': 'hello', 'input2': 'goodbye'},
 'y': {'input': 'hello', 'input2': 'goodbye'}}

In [17]:
chain = RunnableParallel({"x": RunnablePassthrough(), "y": lambda z: z["input2"]})

In [18]:
chain.invoke({"input": "hello", "input2": "goodbye"})

{'x': {'input': 'hello', 'input2': 'goodbye'}, 'y': 'goodbye'}

### Nested Chains

In [19]:
def find_keys_to_uppercase(input: dict):
    output = input.get("input", "not found").upper()
    return output

In [20]:
chain = RunnableParallel({"x": RunnablePassthrough() | RunnableLambda(find_keys_to_uppercase), "y": lambda z: z["input2"]})

In [21]:
chain.invoke({"input": "hello", "input2": "goodbye"})

{'x': 'HELLO', 'y': 'goodbye'}

In [22]:
chain = RunnableParallel({"x": RunnablePassthrough()})

def assign_func(_):
    return 100

def multiply(input):
    return input * 10

In [23]:
chain.invoke({"input": "hello", "input2": "goodbye"})

{'x': {'input': 'hello', 'input2': 'goodbye'}}

In [24]:
chain = RunnableParallel({"x": RunnablePassthrough()}).assign(extra=RunnableLambda(assign_func))

In [25]:
result = chain.invoke({"input": "hello", "input2": "goodbye"})
print(result)

{'x': {'input': 'hello', 'input2': 'goodbye'}, 'extra': 100}


### Combine Multiple Chains

In [26]:
def extractor(input: dict):
    return input.get("extra", "Key not found")

def cupper(upper: str):
    return str(upper).upper()

new_chain = RunnableLambda(extractor) | RunnableLambda(cupper)

In [27]:
new_chain.invoke({"extra": "test"})

'TEST'

In [28]:
final_chain = chain | new_chain
final_chain.invoke({"input": "hello", "input2": "goodbye"})

'100'